<a href="https://colab.research.google.com/github/profteachkids/CHE2064/blob/master/Demo_AdiabaticFlash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries
This section is somewhat intimidating to new users, but modifications are necessary only to access advanced features.

In [1]:
!git clone --depth 1 https://github.com/profteachkids/CHE2064.git &> /dev/null
!pip install DotMap &> /dev/null
import sys
sys.path.insert(1, "/content/CHE2064") #Path to CHE module imports

from dotmap import DotMap
import jax
import jax.numpy as jnp
from jax.config import config
config.update("jax_enable_x64", True) #JAX default is 32bit single precision

from tools.tree_array_transform import VSC 
from tools.trust_ncg import minimize
import tools.che as che

## Dot-access to chemical properties and calculations
ChemCAD data properties and binary interaction parameters can be exported as text files.  A few common chemicals are stored in a Github repository.  Users with ChemCad access can export and access their own data files.  The *che.Props* class parses these files for convenient dot-access in Python calculations.

In [2]:
p = che.Props(['Ethanol','Isopropanol', 'Water'])

In [3]:
print(f'Tc: {p.Tc}')
print(f'Pc: {p.Tc}')
print(f'Vapor Pressure at 432.1 K {p.Pvap(432.1)} Pa')
print(f'NRTL activity coefficients for equimolar mixture {p.NRTL_gamma([1/3,1/3,1/3], 300)} ')

Tc: [513.92 508.31 647.35]
Pc: [513.92 508.31 647.35]


/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


Vapor Pressure at 432.1 K [1220670.06610508 1069634.05615236  600793.6517417 ] Pa
NRTL activity coefficients for equimolar mixture [1.11669551 1.12519412 1.85757473] 


## Adiabatic Flash Calculation
Model mass/energy balance and VLE.  All variables (knowns and unknowns) are in a convenient DotMap structure (c).  The model is written once.

Sum up weighted square deviations for minimization.
 

In [ ]:
def model(c):
    Ltot = c.Ftot - c.Vtot  # What's not in the vapor is in the liquid
    V = c.Vy * c.Vtot # Moles of each component = mole fractions * total moles
    L = c.Lx * Ltot
    F = c.Fz * c.Ftot
    mass_balance = (F - V - L) # Mass balance for each component (vectors!)
    mass_balance = (mass_balance/jnp.sum(F))**2

    # Hmix calculates the enthalpy given the temperature and moles of each
    # component in the vapor and liquid phases
    FH = p.Hmix(nV=0.0, nL=F,   T=c.FT)
    VH = p.Hmix(nV=V,   nL=0.0, T=c.flashT)
    LH = p.Hmix(nV=0.0, nL=L,   T=c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    # Raoults with NRTL activity coefficient correction.  One-liner!
    VLE = (c.Lx  * p.NRTL_gamma(c.Lx,c.flashT)* p.Pvap(c.flashT)
                - c.Vy*c.flashP)
    VLE = (VLE/jnp.sum(c.Vy*c.flashP))**2

    # Square deviations are weighted by their approx. magnitudes and
    # summed for minimization
    return jnp.sum(mass_balance + energy_balance + VLE)

In [ ]:
s=DotMap()
s.Ftot=10
s.Fz = jnp.array([1/3, 1/3, 1/3])
s.FT = 450

s.flashP= 101325
v=DotMap()
v.Vy = tat.Comp(s.Fz)
v.Lx = tat.Comp(s.Fz)
v.flashT = tat.Range(360, 300, s.FT)
v.Vtot = tat.Range(s.Ftot/2, 0., s.Ftot)

In [ ]:
c=VSC(v,s)
modelf = jax.jit(c.transform(model))

In [4]:
x_min, f = tncg.minimize(modelf,c.x, verbosity=1)

In [ ]:
s=DotMap()
s.Ftot=10
s.Fz = jnp.array([1/3, 1/3, 1/3])
s.FT = 450

s.flashP= 101325
v=DotMap()
v.Vy = tat.Comp(s.Fz)
v.Lx = tat.Comp(s.Fz)
v.flashT = tat.Range(360, 300, s.FT)
v.Vtot = tat.Range(s.Ftot/2, 0., s.Ftot)

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [ ]:
c=VSC(v,s)
modelf = jax.jit(c.transform(model))

In [ ]:
x_min, f = tncg.minimize(modelf,c.x, verbosity=1)

0:6, f: 0.046582062928938066
1:5, f: 0.0030325209129333807
2:5, f: 3.8263760766167256e-05
3:5, f: 9.734012881667384e-09
Final results:
f: 7.292860925998015e-16
x: [-0.14989821 -0.10167514 -0.41635314  0.23361766  0.16785475 -0.60540527]


In [ ]:
v_sol = c.xtoc(x_min)
df=tat.todf(v_sol)
df

vector1   vector3                    
              1         1         2         3
FT          450                              
Ftot         10                              
Fz               0.333333  0.333333  0.333333
Lx               0.311418  0.326803  0.361779
Vtot     3.9739                              
Vy               0.366567  0.343236  0.290198
flashP   101325                              
flashT  352.966